In [1]:
from pathlib import Path
from pyraphtory.context import PyRaphtory

In [2]:
pr = PyRaphtory(spout_input=Path('/tmp/lotr.csv'), builder_script=Path('builder.py'),  builder_class='LotrGraphBuilder', mode='batch', logging=True).open()

b'WARNING: sun.reflect.Reflection.getCallerClass is not supported. This will impact performance.\n'
b'14:09:09.100 [io-compute-33] DEBUG com.raphtory.utils.FileUtils$ - Found single file /tmp/lotr.csv matching criteria.\n'
b"14:09:09.101 [io-compute-33] DEBUG com.raphtory.utils.FileUtils$ - Temporary directory '/tmp/raphtory-file-spout/-7254970907570689463' successfully created.\n"
b"14:09:09.102 [io-compute-33] DEBUG com.raphtory.spouts.FileSpout - Spout: Found a new file '/lotr.csv' to process.\n"
b"14:09:09.102 [io-compute-33] DEBUG com.raphtory.spouts.FileSpout - Spout: Attempting to hard link file '/tmp/lotr.csv' -> '/tmp/raphtory-file-spout/-7254970907570689463/lotr.csv'.\n"
b"14:09:09.102 [io-compute-33] INFO  com.raphtory.spouts.FileSpout - Spout: Processing file 'lotr.csv' ...\n"
b'14:09:09.179 [io-compute-33] INFO  com.raphtory.internals.management.Prometheus$ - Prometheus started on port /0:0:0:0:0:0:0:0:9999\n'
b'14:09:09.636 [spawner-akka.actor.default-dispatcher-3] INFO  

In [3]:
from pyraphtory.algo import Vertex, Iterate, Step

In [4]:
PR_LABEL = 'prlabel'

class PGStep1(Step):
    def eval(self, v: Vertex):
        initLabel = 1.0
        v[PR_LABEL] = initLabel
        out_degree = v.out_degree()
        if out_degree > 0:
            msg = initLabel / out_degree
            v.message_outgoing_neighbours(msg)

class PGIterate1(Iterate):
    def __init__(self, iterations: int, execute_messaged_only: bool, damping_factor: float = 0.85):
        super().__init__(iterations, execute_messaged_only)
        self.damping_factor = damping_factor

    def eval(self, v: Vertex):
        current_label = v[PR_LABEL]
        queue = v.message_queue()
        summed_queue = sum(queue)
        new_label = (1 - self.damping_factor) + self.damping_factor * summed_queue
        v[PR_LABEL] = new_label

        out_degree = v.out_degree()

        if out_degree > 0:
            v.message_outgoing_neighbours(new_label / out_degree)

        if abs(new_label - current_label) < 0.00001:
            v.vote_to_halt()


In [5]:
rg = pr.graph()

In [6]:
tracker = rg.at(32674) \
                .past() \
                .step(PGStep1()) \
                .iterate(PGIterate1(iterations=100, execute_messaged_only=False)) \
                .select([PR_LABEL]) \
                .write_to_file("/tmp/jupyter_pagerank")

In [7]:
print(tracker)

In [10]:
tracker.wait_for_job()

In [ ]:
!ls /tmp/jupyter_pagerank/*/partition-0.csv

In [ ]:
tracker.inner_tracker().getJobId()

In [ ]:
!head /tmp/jupyter_pagerank/124901084_925143957368878401/partition-0.csv

In [11]:
tracker.inner_tracker().getJobId()

'124901084_925143957368878401'

In [13]:
!head /tmp/jupyter_pagerank/124901084_925143957368878401/partition-0.csv

32674,Hirgon,0.15000000000000002
32674,Hador,0.24807333518209068
32674,Horn,0.28189681864532734
32674,Galadriel,1.2027625938738855
32674,Isildur,0.15000000000000002
32674,Mablung,0.24960771951147775
32674,Gram,1.2872305970253017
32674,Thingol,0.6812439557732323
32674,Celebrían,0.17042791413738384
32674,Gamling,0.2079389980212792
